<a href="https://colab.research.google.com/github/M-1620/pic1/blob/main/Critical_AI_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-dartmouth > /dev/null
!pip install faiss-cpu > /dev/null

from langchain_dartmouth.llms import ChatDartmouth, DartmouthLLM
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_dartmouth.embeddings import DartmouthEmbeddings

from langchain.docstore.document import Document
from langchain_dartmouth.retrievers.document_compressors import DartmouthReranker

from langchain_community.vectorstores import FAISS
import faiss
import json

from sentence_transformers import SentenceTransformer

import os
import numpy as np

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
os.environ["DARTMOUTH_API_KEY"] = "uxAAdJGo24JNT3B1STbF5YiMlG2waTCZ7vinyjfI8538MCO0jbxseXT4PH8KKOMt8VQHnMVEnfbAogUWu4soN3vnob7pXD2A6X5NRP7ymqv6ACY4n9xw3bfhsS90HCzBvqv7ZJyGrFxWxNNwwDDs4t46NB8Y7fiEHSx7nYOePqnJPcY2ChotC4yLnXuWNBNy6ML7cGLNYTrWrvDL8qWlFedEyFBsr8cvuudc4vgsMmfHP4vRwrfqhV1puL2HN0pTlzAnpz2fJYe7dxGeya1380JYFYtETKeYBPVuoSvPP80UkYWcx16YnWckQbQKodfXP54LbZ8BK5KGfPTl49cXOIFV"

In [ ]:
def format_prompt(prompt):
    """
    This function will format a prompt into what is needed for LangChain to produce ChatML.
    Args:
       prompt: the text to be embedded as human prompt.
    """
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=prompt),
    ]
    return messages

In [ ]:
max_new_tokens = 1024
top_p = 0.95 # If set to < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
temperature = .95 #  Strictly positive float value used to modulate the logits distribution. A value smaller than 1 decreases randomness (and vice versa), with 0 being equivalent to shifting all probability mass to the most likely token
repetition_penalty = None

system_prompt = "Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity."
kwargs = dict()

llm_chat = ChatDartmouth(model_name="llama-3-1-8b-instruct",
                    temperature = temperature,
                    top_p = top_p,
                    max_tokens = max_new_tokens,
                    model_kwargs=kwargs)

In [ ]:
# download LayupList data
!wget -O /content/old_reviews.json https://raw.githubusercontent.com/jeddobson/ENGL64.05-22F/refs/heads/main/data/LayupList/old_reviews.json

--2024-11-22 17:18:54--  https://raw.githubusercontent.com/jeddobson/ENGL64.05-22F/refs/heads/main/data/LayupList/old_reviews.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18938407 (18M) [text/plain]
Saving to: ‘/content/old_reviews.json’

/content/old_review 100%[===================>]  18.06M  --.-KB/s    in 0.07s   

2024-11-22 17:18:56 (254 MB/s) - ‘/content/old_reviews.json’ saved [18938407/18938407]



In [ ]:
# open older format reviews and extract comments
reviews = json.loads(open("old_reviews.json").read())
reviews_text= [r["comments"]["oldReview"] for r in reviews if 'comments' in r]
# how many did we find?
print("found {0} reviews".format(len(reviews_text)))

found 29000 reviews


In [ ]:
# load a smaller embedding model that will quickly embed all our documents
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Create document embeddings with embedding model
doc_embeddings = embedding_model.encode(reviews_text)

In [ ]:
# display number of documents and embedding width
doc_embeddings.shape

(29000, 384)

In [ ]:
# Build FAISS index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

# this will retrieve five closest neighbors using document similarity
def retrieve_documents(query, k=5):
    query_embedding = embedding_model.encode([query])[0]
    distances, indices = index.search(np.array([query_embedding]), k)
    return [reviews_text[i] for i in indices[0]]

NameError: name 'doc_embeddings' is not defined

In [ ]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from langchain_dartmouth.llms import ChatDartmouth

# Set parameters for the language model
max_new_tokens = 1024
top_p = 0.95
temperature = 0.95
repetition_penalty = None

# System prompt for the language model
system_prompt = "Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity."

# Initialize the language model
llm_chat = ChatDartmouth(
    model_name="llama-3-1-8b-instruct",
    temperature=temperature,
    top_p=top_p,
    max_tokens=max_new_tokens,
)

# Load the policies from the JSON file (adjust the path as needed)
with open("/content/sample_data/policy.json") as f:
    reviews = json.load(f)

# Collect all relevant text from the JSON structure for New England states (Connecticut, Rhode Island, Massachusetts)
reviews_text = []
for policy in reviews:
    if policy["state"] in {"Connecticut", "Rhode Island", "Massachusetts"}:  # Filter for New England states
        # Append goal and description
        reviews_text.append(policy["goal"])
        reviews_text.append(policy["description"])
        # Collect major activities text
        for activity in policy["majorActivities"]:
            reviews_text.append(activity["description"])
            reviews_text.extend(activity["activities"])  # Add all activities

# Count the total number of loaded reviews
print("Found {0} policies and activities".format(len(reviews_text)))

# Load a smaller embedding model for document embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create document embeddings
doc_embeddings = embedding_model.encode(reviews_text)

# Build a FAISS index with the document embeddings
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

# Function to retrieve documents based on a query
def retrieve_documents(query, k=5):
    query_embedding = embedding_model.encode([query])[0]
    distances, indices = index.search(np.array([query_embedding]), k)
    return [reviews_text[i] for i in indices[0]]


Found 206 policies and activities


In [ ]:
# Example query for New England policies
query = "what is vermont doing in relation to act 250"

# Retrieve relevant documents for the query
retrieved_docs = retrieve_documents(query, k=10)

# Combine query with the context from the retrieved documents
context = query + "\n" + "\n".join(retrieved_docs)

# Function to format the context prompt for the model
def format_prompt(context):
    return f"{system_prompt}\n{context}"

# Generate output from the language model
output = llm_chat.invoke(format_prompt(context))
print(output.content)

Act 250 is a land use and development law in Vermont, USA. The law requires a thorough environmental review for development projects that involve the use of 10 or more acres of land. The goal of Act 250 is to ensure that development projects are environmentally friendly and do not harm Vermont's natural resources.

In relation to Act 250, Vermont is taking the following steps:

1. **Amending and Revising the Law**: Vermont has passed legislation to amend Act 250, including updating the environmental review process and imposing stricter regulations on development projects. The aim is to balance economic growth with environmental protection.
2. **Enhancing Transparency**: Vermont has increased transparency in the Act 250 process, making it easier for the public to access information about development projects and participate in the review process.
3. **Streamlining the Process**: Efforts are being made to streamline the Act 250 process, reducing bureaucracy and speeding up the review pro

In [ ]:
query = "What are policies benefiting disabled people in Massachusetts?"

# Retrieve relevant documents for our query
retrieved_docs = retrieve_documents(query,k=10)

# Join query with context.
context = query + "\n" + "\n".join(retrieved_docs)

In [ ]:
# Generate!
output = llm_chat.invoke(format_prompt(context))
print(output.content)

In [ ]:
# Display context sent to LLM
context

In [ ]:
query = "What are some of RIBLIA caucus’ goals?"

# Retrieve relevant documents for our query
retrieved_docs = retrieve_documents(query,k=10)

# Join query with context.
context = query + "\n" + "\n".join(retrieved_docs)

In [ ]:
# Generate!
output = llm_chat.invoke(format_prompt(context))
print(output.content)